# Data Exploration

Here we will not treat the binaries like a black box bot rather use our knowledge of machine language in general
and particular ISA's to analyze the data set.
We will gather some general features common to most or all ISA's and deep dive into a few.

Online disassembler with > 60 archs
https://onlinedisassembler.com/static/home/index.html

https://en.wikibooks.org/wiki/X86_Disassembly/Disassemblers_and_Decompilers#Linux_Disassemblers

## Questions
###1
Is it possible to have a binary snippet from the game represent a valid machine instruction in two distinct ISA's

Experiment: Using https://onlinedisassembler.com/odaweb/Ufx0uq75/0 Select arch = alphav5 or alphav6 (correct target)
00 00 00 00 00 01 7a 52 00 04 78 1a 01 1b 0d 1e 00 18 00 00 00 18 00 00 00 00 00 00 00 8c 00 00 00 00 43 0e 10 41 9a 02 5e da 0e 00 00 03 00 e3 43 00 00 3d 24 00 00 61 b0 00 00 42 20 04 00 42
Output has no "unknown" translations
Endianness does not affect output
Select arm
Output has no "unknown" translations
i386-x86_64:
  27 lines
  unusual:
    0e: (bad)
    1b0d1e001800: apparently 6 word instruction?
mips:
  unusual: several items are just copied from hex string to assembly.  Perhaps the IDK for this arch?
  notes: all lines are 8 hex chars
  029a4110: 0x29a4110
mips16:
  unusual: three jal<x> instructions of 8 hex chars, the rest are 4 hex chars.
sh4:
  notes: all lines are 4 hex chars
  unusual:
    0018: sett; single sintruction no args?
  

# sparc

Overview http://moss.csc.ncsu.edu/~mueller/codeopt/codeopt00/notes/sparc.html
## RISC
### 3 types of instruction
#### ALU + shift
opcode reg1,reg2,reg3    !reg1 op reg2    -> reg3
opcode reg1,const13,reg3 !reg1 op const13 -> reg3
#### Load, Store
opcode [reg1+reg2],reg3
opcode [reg1+const13],reg3
#### Branch
opcode address


32 registers, 32 fp registers, 32 coprocessor registers, 10 special registers, 4 unary ops, 

All load and store begin with 11



Examples
http://www.cs.unm.edu/~maccabe/classes/341/labman/node9.html
Example: Hand assemble the instruction:
ldd     [%r4+%r7], %r11
Because this instruction uses two registers in the address specification, it is encoded using the first format shown in Figure 9.1. As such, we must determine the values for the rd, op tex2html_wrap_inline5419 , rs tex2html_wrap_inline5421 , and rs tex2html_wrap_inline5423 fields. The following table summarizes these encodings:

tabular2256

These encodings lead to the following machine instruction:

picture2263

That is, 1101 0110 0001 1001 0000 0000 0000 0111 in binary, or 0xD6190007.

If the assembly language instruction only uses a single register in the address specification (e.g., register indirect addressing), the register is encoded in one of the source register fields (i.e., sr tex2html_wrap_inline5421 or sr tex2html_wrap_inline5423 ) while %r0 is encoded in the other. It doesn't matter which field holds the register specified in the assembly language instruction and which field holds the encoding for %r0. However, isem-as encodes %r0 in sr tex2html_wrap_inline5423 .

Example: Hand assemble the instruction:
ldub     [%r23], %r19
Because this instruction uses registers in the address specification, it is encoded using the first format shown in Figure 9.1. As such, we must determine the values for the rd, op tex2html_wrap_inline5419 , rs tex2html_wrap_inline5421 , and rs tex2html_wrap_inline5423 fields. The following table summarizes these encodings:

tabular2311

These encodings lead to the following machine instruction:

picture2318

That is, 1110 0110 0000 1101 1100 0000 0000 0000 in binary, or 0xE60DC000.

# ARM

32 bits
instruction type, condition, operands

some of the status bits are stored in bits 26..31 of R15

### Groupings 5.5

Group one

MOV, ADD etc. 1 s-cycle. If <rhs> contains a shift count in a register (i.e. not an immediate shift), add 1 s-cycle. If <dest> is R15, add 1 s + 1 n-cycle.

Group one A

MUL, MLA. 1 s + 16 i- cycles worst-case.

Group two

LDR. 1 s + 1 n + 1 i-cycle. If <dest> is R15, add 1 s + 1n-cycle.

STR. 2 n-cycles.

Group three

LDM. (regs-1) s + 1 n + 1 i-cycle. Regs is the number of registers loaded. Add 1 s + 1 n-cycles if R15 is loaded.

STM. 2 n + (regs-1) s-cycles.

Group four

B, BL. 2 s + 1 n-cycles.

Group five

SWI. 2 s + 1 n-cycles.

Hex to ARM and back converter
http://armconverter.com/

All arithmetic functions have opcode 0x -> 000000


https://stackoverflow.com/questions/11785973/converting-very-simple-arm-instructions-to-binary-hex

mov r0, r0 should go like this:

1110 00 0 0 1101 0000 0000 00000000

# MIPS

### Groupings 5

https://en.wikibooks.org/wiki/MIPS_Assembly/Instruction_Formats

Converter assembly <-> hex/binary https://www.eg.bucknell.edu/~csci320/mips_web/


In [ ]:
opcode_mapping = """
R:
 description: all the data values used by the instruction are located in registers.
 opcode_pattern: 
  opcode: 
    bits: 6
  rs: 
    bits: 5
    indices: [21,25]
  rt: 
    bits: 5
    indices: [16, 20]
  rd: 
    bits: 5
  shift: 
    bits: 6
  funct: 
    bits: 6

I:
  description: the instruction must operate on an immediate value and a register value. 
  opcode_pattern:
    opcode: 
      bits: 6
    rs: 
      bits: 5
    rt: 
      bits: 5
    IMM: 
      bits: 16
J:
  description: jump instructions
  opcode_pattern:
    opcode: 
      bits: 6
    pseudo_addr: 
      bits: 26
      description: 
       - he two least significant bits are removed, and the 4 most significant bits are removed, 
       - and assumed to be the same as the current instruction's address.
      
FR:
  description: instruction uses data only from floating-point registers
 opcode_pattern: 
  opcode: 
    bits: 6
  fmt: 
    bits: 5
  ft: 
    bits: 5
  fs: 
    bits: 5
  fd: 
    bits: 6
  funct: 
    bits: 6  
  
FI:
  description: the instruction must operate on an immediate value and a register value.
  opcode_pattern:
    opcode: 
      bits: 6
    fmt: 
      bits: 5
    ft: 
      bits: 5
    IMM: 
      bits: 16    
"""



# x86_64

assembly <-> hex https://defuse.ca/online-x86-assembler.htm

Special
Stack Pointer, Base Pointer

In [1]:
base64_encoded = u'GAAAABgAAAAAAAAAqAAAAABIDhBHmgJa2g4AAAAAQaAAAD0kAABetxEE/0cCAOJDAAAQIgAAQbAAAF0kAABCoA=='

# SH4
http://www.shared-ptr.com/sh_insns.html

In [11]:
import base64
import binascii

In [6]:
hx = base64.b64decode(base64_encoded)
repr(hx)

'\'\\x18\\x00\\x00\\x00\\x18\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\xa8\\x00\\x00\\x00\\x00H\\x0e\\x10G\\x9a\\x02Z\\xda\\x0e\\x00\\x00\\x00\\x00A\\xa0\\x00\\x00=$\\x00\\x00^\\xb7\\x11\\x04\\xffG\\x02\\x00\\xe2C\\x00\\x00\\x10"\\x00\\x00A\\xb0\\x00\\x00]$\\x00\\x00B\\xa0\''

In [7]:
len(hx)

64

In [8]:
hx[0]

'\x18'

In [48]:
binascii.hexlify('00=$')

'30303d24'

In [42]:
bins = [bin(long(binascii.hexlify(x), 16)) for x in hx]
print(bins[0:6])

['0b11000', '0b0', '0b0', '0b0', '0b11000', '0b0']


In [44]:
bins = [x + '0'*(2 + 8 - len(x)) for x in bins]
print(bins[0:6])

['0b11000000', '0b00000000', '0b00000000', '0b00000000', '0b11000000', '0b00000000']


In [45]:
len(bins)/4

16

In [47]:
2**32/10**6

4294

In [49]:
import etl

In [50]:
data_set = etl.load('ml_challenge/4096_512_512_6f9ef7d8.json')

In [55]:
orig_X_train, orig_Y_train, orig_train_targets = data_set['train']['binary_data'], data_set['train']['answers'], data_set['train']['targets']
orig_X_dev, orig_Y_dev, orig_dev_targets = data_set['dev']['binary_data'], data_set['dev']['answers'], data_set['dev']['targets']
orig_X_test, orig_Y_test, orig_test_targets = data_set['test']['binary_data'], data_set['test']['answers'], data_set['test']['targets']

In [57]:
print(orig_X_train[0:6])
print(orig_Y_train[0:6])
print(orig_train_targets[0:6])

[u'GAAAABgAAAAAAAAAqAAAAABIDhBHmgJa2g4AAAAAQaAAAD0kAABetxEE/0cCAOJDAAAQIgAAQbAAAF0kAABCoA==', u'AAAAAAABelIABHgaARsNHgAYAAAAGAAAAAAAAACMAAAAAEMOEEGaAl7aDgAAAwDjQwAAPSQAAGGwAABCIAQAQg==', u'AOUBIRjRGdMSYRnUEB9iYRjRMiEyITIhCtMCITJjFtAyISIhYmMA5jIhIiFxUiIhclIiIQtACQAA4AR/Jk/2aQ==', u'g/gCfui4AAAAAOsTSGPQixSVAAAAAIkVAAAAAIPAAYXAfum4AAAAAOsTSGPQD7aSAAAAAIkVAAAAAIPAAYP4AQ==', u'ZFAQ8GBYENEapyr//1AgEABYENEaWCAQAKcu//KnRP/sWMDwYKdIAAFYMNEaWBDRFg3hpzgA/xQyWEDRGkEg8A==', u'sHynOA/hQTMgAEEzEACSjjAAWBCwfKcaAAFQELB8WBCwfKceAAKnxP/oWBBQ7lgQEAASEaeEAAanGAABp/QABA==']
[u'alphaev56', u'alphaev56', u'sh4', u'x86_64', u's390', u's390']
[[u'alphaev56', u'arm', u'm68k', u'mips', u'x86_64', u'xtensa'], [u'alphaev56', u'arm', u'avr', u'm68k', u'mipsel', u'powerpc'], [u'avr', u'm68k', u'mips', u'mipsel', u'sh4', u'sparc'], [u'arm', u'm68k', u'mips', u'mipsel', u'sh4', u'x86_64'], [u'arm', u'avr', u'm68k', u's390', u'x86_64', u'xtensa'], [u'alphaev56', u'arm', u'm68k', u's390', u'sh4', u'x86_64']]


In [58]:
hex_X_train, train_misfits = etl.hex_data(orig_X_train)
hex_X_dev, dev_misfits = etl.hex_data(orig_X_dev)
hex_X_test, test_misfits = etl.hex_data(orig_X_test)

In [59]:
print(hex_X_train[0:6])

[['18' '00' '00' '00' '18' '00' '00' '00' '00' '00' '00' '00' 'a8' '00'
  '00' '00' '00' '48' '0e' '10' '47' '9a' '02' '5a' 'da' '0e' '00' '00'
  '00' '00' '41' 'a0' '00' '00' '3d' '24' '00' '00' '5e' 'b7' '11' '04'
  'ff' '47' '02' '00' 'e2' '43' '00' '00' '10' '22' '00' '00' '41' 'b0'
  '00' '00' '5d' '24' '00' '00' '42' 'a0']
 ['00' '00' '00' '00' '00' '01' '7a' '52' '00' '04' '78' '1a' '01' '1b'
  '0d' '1e' '00' '18' '00' '00' '00' '18' '00' '00' '00' '00' '00' '00'
  '00' '8c' '00' '00' '00' '00' '43' '0e' '10' '41' '9a' '02' '5e' 'da'
  '0e' '00' '00' '03' '00' 'e3' '43' '00' '00' '3d' '24' '00' '00' '61'
  'b0' '00' '00' '42' '20' '04' '00' '42']
 ['00' 'e5' '01' '21' '18' 'd1' '19' 'd3' '12' '61' '19' 'd4' '10' '1f'
  '62' '61' '18' 'd1' '32' '21' '32' '21' '32' '21' '0a' 'd3' '02' '21'
  '32' '63' '16' 'd0' '32' '21' '22' '21' '62' '63' '00' 'e6' '32' '21'
  '22' '21' '71' '52' '22' '21' '72' '52' '22' '21' '0b' '40' '09' '00'
  '00' 'e0' '04' '7f' '26' '4f' 'f6' '69']
 ['83' 

In [60]:
import pandas as pd

In [67]:
dft = pd.DataFrame(zip(hex_X_train, orig_Y_train))

In [64]:
dft.describe()

,0,1
count,4096,4096
unique,4096,12
top,"[02, 04, a2, 44, 22, 00, e2, 73, 00, 00, fd, 2...",x86_64
freq,1,362


Observe that there are 4096 unique snippets, so no collisions between archs here.  
There are 16 ** (len(snippet) * (2 digits per 'word')) = 16 ** 128 possibilities

In [68]:
dft.columns = ['hex_list', 'arch']

In [69]:
dft.head()

,hex_list,arch
0,"[18, 00, 00, 00, 18, 00, 00, 00, 00, 00, 00, 0...",alphaev56
1,"[00, 00, 00, 00, 00, 01, 7a, 52, 00, 04, 78, 1...",alphaev56
2,"[00, e5, 01, 21, 18, d1, 19, d3, 12, 61, 19, d...",sh4
3,"[83, f8, 02, 7e, e8, b8, 00, 00, 00, 00, eb, 1...",x86_64
4,"[64, 50, 10, f0, 60, 58, 10, d1, 1a, a7, 2a, f...",s390


For each ISA find the frequencies of 2-char 'words' like '4d', 'cf'

In [89]:
import collections
from __future__ import division

In [125]:
def build_counts(hex_lists, archs):
    """
    hex_lists: list of 64 item list of 32-bit hex items
    archs: list of arch strings
    @returns: 12 counters one for each arch
    """
    counters = dict(zip(etl.SUPPORTED_ARCHITECTURES, [collections.Counter() for 
                                                      i in range(len(etl.SUPPORTED_ARCHITECTURES))]))
    for h, arch in zip(hex_lists, archs):
        counters[arch].update(h)
    return counters
        

In [126]:
counters = build_counts(hex_X_train, orig_Y_train)
print(counters.keys())

['s390', 'sh4', 'x86_64', 'xtensa', 'avr', 'powerpc', 'alphaev56', 'sparc', 'm68k', 'mips', 'mipsel', 'arm']


In [133]:
def freq_stats(counters):

    stats = {}
    for arch in etl.SUPPORTED_ARCHITECTURES:
        total_instructions = 0
        unique_instructions = 0
        repeated_instructions = 0
        repeats = []
        for instruction, count in counters[arch].most_common():
            total_instructions += count
            unique_instructions += 1
            if count > 1:
                repeats.append(instruction)
                repeated_instructions += 1
        stats[arch] = {'vocab_size': unique_instructions,
                       'repeated_instructions': repeated_instructions,
                       #'repeats': set(repeats),
                       'unique_instructions': unique_instructions,
                       'total_instructions': total_instructions,
                       'percent_repeated': (repeated_instructions/unique_instructions)*100.0,
                       'most_common_20': set([x[0] for x in counters[arch].most_common(20)]),
                       'least_common_20': set([x[0] for x in counters[arch].most_common()[:-20-1:-1]])}
    return stats
                
        

In [134]:
fstats = freq_stats(counters)
print(fstats)

{'s390': {'vocab_size': 243, 'percent_repeated': 90.94650205761316, 'least_common_20': set(['75', '77', '6b', 'ab', 'ad', '37', '9d', 'dc', '99', '61', 'a3', '63', 'e9', 'b1', '2b', '83', 'c3', '3b', '5f', '4b']), 'repeated_instructions': 221, 'unique_instructions': 243, 'total_instructions': 21376, 'most_common_20': set(['f0', '10', '00', '01', '20', '58', '0d', 'ff', '18', '30', '28', '50', '40', '07', '60', 'b0', 'a7', '0e', 'd0', 'd1'])}, 'sh4': {'vocab_size': 255, 'percent_repeated': 98.4313725490196, 'least_common_20': set(['aa', 'ac', 'ab', 'ae', 'ad', '9b', '9e', 'bf', '9f', '99', 'a5', 'a4', 'c6', 'a8', 'c0', 'cd', 'b8', 'b9', '8e', 'ba']), 'repeated_instructions': 251, 'unique_instructions': 255, 'total_instructions': 23104, 'most_common_20': set(['02', '00', '12', '21', '22', '0c', 'ff', '32', '09', '40', '0b', '61', '62', '63', '72', '71', '01', 'd2', 'e3', 'd1'])}, 'x86_64': {'vocab_size': 255, 'percent_repeated': 99.6078431372549, 'least_common_20': set(['6a', '26', '6b',

Conclusion: At the granularity of 32-bit instructions only ~ 1% of instructions are in the repetition set of an arch.
The total instructions seen by each counter is total (samples/num_archictectures) * (instructions/sample)

In [110]:
4096/12 * 64

21845.333333333332

In [141]:
def stop_words(freq_stats):
    union = set([])
    wildspace = {}
    # could choose any arch to initialize intersection
    intersection = set(freq_stats['sparc']['most_common_20'])
    for arch in freq_stats:
        intersection.intersection(freq_stats[arch]['most_common_20'])
        others_union = set([])
        union = union.union(freq_stats[arch]['most_common_20'])
        others = set(freq_stats.keys()) - set([arch])
        for other in others:
            others_union = others_union.union(freq_stats[other]['most_common_20'])
        wildspace_item = freq_stats[arch]['most_common_20'].difference(others_union)
        wildspace[arch] = wildspace_item
    return intersection, wildspace

In [142]:
stop_words, wildspace = stop_words(fstats)

In [144]:
print(len(stop_words), stop_words)

(20, set(['bf', '03', '60', 'a0', 'ff', '82', 'c2', '80', 'e0', '84', 'c4', '10', '00', '01', '20', '07', '05', '08', '40', 'c6']))


In [145]:
print(wildspace)

{'s390': set(['58', 'a7', '50']), 'sh4': set(['12', '32', '0b', '61', '62', '72', '71', 'd2']), 'x86_64': set(['48', '8b', '45', 'eb', 'f8', 'c7']), 'xtensa': set(['99', '98', '88', 'a9']), 'avr': set(['ef', '2f', 'b7', '1f', '92', '95']), 'powerpc': set(['3d', '38', '7d', '14', '49']), 'alphaev56': set(['47']), 'sparc': set(['bf', '82', 'c2', 'c6', '84', 'c4']), 'm68k': set(['2e', '4a', 'fc', '4e']), 'mips': set([]), 'mipsel': set(['af']), 'arm': set(['e5', 'e2', 'e1', '9f'])}


In [150]:
{ orig_Y_train[i]: " ".join(hex_X_train[i]) for i in range(5)}

{u'alphaev56': '00 00 00 00 00 01 7a 52 00 04 78 1a 01 1b 0d 1e 00 18 00 00 00 18 00 00 00 00 00 00 00 8c 00 00 00 00 43 0e 10 41 9a 02 5e da 0e 00 00 03 00 e3 43 00 00 3d 24 00 00 61 b0 00 00 42 20 04 00 42',
 u's390': '64 50 10 f0 60 58 10 d1 1a a7 2a ff ff 50 20 10 00 58 10 d1 1a 58 20 10 00 a7 2e ff f2 a7 44 ff ec 58 c0 f0 60 a7 48 00 01 58 30 d1 1a 58 10 d1 16 0d e1 a7 38 00 ff 14 32 58 40 d1 1a 41 20 f0',
 u'sh4': '00 e5 01 21 18 d1 19 d3 12 61 19 d4 10 1f 62 61 18 d1 32 21 32 21 32 21 0a d3 02 21 32 63 16 d0 32 21 22 21 62 63 00 e6 32 21 22 21 71 52 22 21 72 52 22 21 0b 40 09 00 00 e0 04 7f 26 4f f6 69',
 u'x86_64': '83 f8 02 7e e8 b8 00 00 00 00 eb 13 48 63 d0 8b 14 95 00 00 00 00 89 15 00 00 00 00 83 c0 01 85 c0 7e e9 b8 00 00 00 00 eb 13 48 63 d0 0f b6 92 00 00 00 00 89 15 00 00 00 00 83 c0 01 83 f8 01'}

u'alphaev56'